In [1]:
# Quiz 1


# Quiz 2 
Estudiante: Santiago Segura

Código: 201617653

Resolución  quiz 

In [70]:
#Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

Leyendo archivo dataset


In [3]:
data_read=pd.read_csv('insurance.csv')

In [4]:
data_read

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


Haciendo rápida exploración de datos 

In [5]:
no_duplicados=data_read.drop_duplicates() #Se eliminan datos no duplicados

In [8]:
no_duplicados.shape # Se obtiene solo un dato duplicado

(1337, 7)

Revisando si hay valores nulos en alguna columna

In [9]:
no_duplicados.isna().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

Como no hay duplicados se procede a hacer la tranformación de los datos categoricos.

In [11]:
no_duplicados['sex'].value_counts()

male      675
female    662
Name: sex, dtype: int64

In [13]:
gender_encode= preprocessing.LabelEncoder()
gender_encode.fit(no_duplicados['sex'])

LabelEncoder()

In [19]:
no_duplicados['sex_enc']=gender_encode.transform(no_duplicados['sex'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
smoke_encode=preprocessing.LabelEncoder()
smoke_encode.fit(no_duplicados['smoker'])

LabelEncoder()

In [24]:
no_duplicados['smoker_enc']=smoke_encode.transform(no_duplicados['smoker'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
region_encode = pd.get_dummies(no_duplicados['region'], prefix='region_')

In [36]:
no_duplicados=pd.concat([no_duplicados, region_encode], axis = 1)

Después de haber transformado las variables categoricas, se dejan las variables transformadas para empezar a trabajar  el problema de regresión

In [38]:
data=no_duplicados.drop(['sex','smoker','region'],axis=1)

In [40]:
data.head()

,age,bmi,children,charges,sex_enc,smoker_enc,region__northeast,region__northwest,region__southeast,region__southwest
0,19,27.900,0,16884.92400,0,1,0,0,0,1
1,18,33.770,1,1725.55230,1,0,0,0,1,0
2,28,33.000,3,4449.46200,1,0,0,0,1,0
3,33,22.705,0,21984.47061,1,0,0,1,0,0
4,32,28.880,0,3866.85520,1,0,0,1,0,0


In [43]:
X=data.drop('charges',axis=1)
Y=data['charges']

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42)

In [48]:
X_train.shape

(1002, 9)

In [49]:
X_test.shape

(335, 9)

In [51]:
#Estandarizando las variables X
scaler_x=preprocessing.StandardScaler().fit(X_train)
X_train_norm=scaler_x.transform(X_train)
X_test_norm=scaler_x.transform(X_test)

Entrenando el primer modelo sin regularización 

In [53]:
lin_reg=LinearRegression()
lin_reg.fit(X_train_norm,y_train)

LinearRegression()

In [59]:
y_predict_train=lin_reg.predict(X_train_norm)
y_predict_test=lin_reg.predict(X_test_norm)

Revisando errores

In [64]:
#training
mae_train=mean_absolute_error(y_train, y_predict_train)
rmse_train=np.sqrt(mean_squared_error(y_train,y_predict_train))
mae_test=mean_absolute_error(y_test, y_predict_test)
rmse_test=np.sqrt(mean_squared_error(y_test,y_predict_test))

print('El MAE para entrenamiento es {}, en test es {} '.format(mae_train,mae_test))

print('El RMS para entrenamiento es {}, en test es {} '.format(rmse_train,rmse_test))

El MAE para entrenamiento es 4207.729191870496, en test es 4069.039466161753 
El RMS para entrenamiento es 6097.720119601683, en test es 5940.0271692232745 


I. Como se puede ver en este caso el error en entrenamiento, es un eror de 6.097 dolares. Que en este caso es un error  no muy alto pero que se puede mejorar. Para el caso de test es un valor muy parecido también, de hecho disminuye para el caso de test.


II. La diferencia entre los errores de entrenamiento y test no son muy altos por lo que no se puede evidenciar que haya oerfitting. Un caso de overfitting sería en donde el error en entrenamiento sea mucho más bajon que en el de test, esto significaría que en el entrenamiento el modelo se ajusto mucho a los datos de entrenamiento.

In [66]:
lin_reg.score(X_train_norm,y_train) #Entrenamiento

0.7297491098061568

In [67]:
lin_reg.score(X_test_norm,y_test) #Test

0.7959403124253077

# Punto 2

Se selecciona un polinomio de grado 2 y grado 6, se procede a hacer la tansformacion

In [71]:
poly_2=PolynomialFeatures(degree=2, include_bias=False)
poly_6=PolynomialFeatures(degree=6, include_bias=False)

In [79]:
# Valores de x con grado 2
X_train_2=poly_2.fit_transform(X_train_norm)
X_test_2=poly_2.fit_transform(X_test_norm)
X_train_2.shape

(1002, 54)

In [80]:
# Valores de x con grado 6
X_train_6=poly_6.fit_transform(X_train_norm)
X_test_6=poly_6.fit_transform(X_test_norm)
X_train_6.shape

(1002, 5004)

In [99]:
#variando el valor de alpha para regularizacion ridge 
alphas=[0.001,0.1,1,10]
print('Para polinomio de grado 2')
for i in alphas:

  ridge_reg=Ridge(alpha=i,solver='cholesky')
  ridge_reg.fit(X_train_2,y_train)
  y_train_ridge=ridge_reg.predict(X_train_2)
  y_test_ridge=ridge_reg.predict(X_test_2)
  mae_train=mean_absolute_error(y_train_ridge, y_train)
  rmse_train=np.sqrt(mean_squared_error(y_train_ridge,y_train))
  mae_test=mean_absolute_error(y_test, y_test_ridge)
  rmse_test=np.sqrt(mean_squared_error(y_test,y_test_ridge))
  print('Para alpha {}'.format(i))
  print('El MAE para entrenamiento es {}, en test es {} '.format(mae_train,mae_test))
  print('El RMS para entrenamiento es {}, en test es {} '.format(rmse_train,rmse_test))
  print('\n')

Para polinomio de grado 2
Para alpha 0.001
El MAE para entrenamiento es 2885.722687832908, en test es 2856.601795934719 
El RMS para entrenamiento es 4764.553583667883, en test es 4694.264812852181 


Para alpha 0.1
El MAE para entrenamiento es 2885.8197192928637, en test es 2856.697965413166 
El RMS para entrenamiento es 4764.553634252683, en test es 4694.285078916484 


Para alpha 1
El MAE para entrenamiento es 2886.701316806681, en test es 2857.5718199721623 
El RMS para entrenamiento es 4764.558632626487, en test es 4694.476183490719 


Para alpha 10
El MAE para entrenamiento es 2895.701034057508, en test es 2866.3081199338853 
El RMS para entrenamiento es 4765.048627406208, en test es 4697.052035911004 




In [98]:
#variando el valor de alpha para regularizacion ridge 
alphas=[0.001,0.1,1,10]
print('Para polinomio de grado 6')
for i in alphas:

  ridge_reg=Ridge(alpha=i,solver='cholesky')
  ridge_reg.fit(X_train_6,y_train)
  y_train_ridge=ridge_reg.predict(X_train_6)
  y_test_ridge=ridge_reg.predict(X_test_6)
  mae_train=mean_absolute_error(y_train_ridge, y_train)
  rmse_train=np.sqrt(mean_squared_error(y_train_ridge,y_train))
  mae_test=mean_absolute_error(y_test, y_test_ridge)
  rmse_test=np.sqrt(mean_squared_error(y_test,y_test_ridge))
  print('Para alpha {}'.format(i))
  print('El MAE para entrenamiento es {}, en test es {} '.format(mae_train,mae_test))
  print('El RMS para entrenamiento es {}, en test es {} '.format(rmse_train,rmse_test))
  print('\n')

Para polinomio de grado 6
Para alpha 0.001
El MAE para entrenamiento es 1543.1583356939482, en test es 114677.19475176786 
El RMS para entrenamiento es 2794.53042244029, en test es 411240.5043626642 


Para alpha 0.1
El MAE para entrenamiento es 1585.096131439866, en test es 22984.700719226475 
El RMS para entrenamiento es 2823.5616482930513, en test es 74382.36511853247 


Para alpha 1
El MAE para entrenamiento es 1627.7878842407895, en test es 13777.00706903319 
El RMS para entrenamiento es 2860.865249173887, en test es 36752.04985738337 


Para alpha 10
El MAE para entrenamiento es 1701.072623126765, en test es 8025.745796768747 
El RMS para entrenamiento es 3009.7427565052208, en test es 19068.893295629918 




i. ¿ Fue posible mejorar el error? Hiper-parámetros que tiene el modelo que produce menor error

Sí. Si se comparan los RMS y MAE de los casos polinomicos y grado dos con el modelo realizado para solo regresión lineal se puede observar que diminuyeron en los modelos para grado 2. Para los de grado 6 se puede observar que en el caso de los errores en test aumentan en comparacion a los de grado 2 y la regresión lineal.

Los errores más bajos se obtienen cuando el modelo tiene grado 2, y alpha de 0.001

El MAE para entrenamiento es 2885.722687832908, en test es 2856.601795934719 
El RMS para entrenamiento es 4764.553583667883, en test es 4694.264812852181

II. Atributos más importantes para hacer predicción

In [103]:
lin_reg.coef_

array([3422.72524125, 1933.67138975,  627.41190011,  -11.72446292,
       9317.08208074,  202.74292671,   18.2473516 , -177.53891735,
        -33.1868679 ])

In [105]:
X_train

,age,bmi,children,sex_enc,smoker_enc,region__northeast,region__northwest,region__southeast,region__southwest
763,27,26.03,0,1,0,1,0,0,0
1079,63,33.66,3,1,0,0,0,1,0
178,46,28.90,2,0,0,0,0,0,1
287,63,26.22,0,0,0,0,1,0,0
1290,38,19.95,2,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
1096,51,34.96,2,0,1,1,0,0,0
1131,27,45.90,2,1,0,0,0,0,1
1295,20,22.00,1,1,0,0,0,0,1
861,38,28.00,3,0,0,0,0,0,1


Como se puede ver de los coeficientes de la regresión, los qque más peso tienen son smoker,edad y bmi. 